It cleans up  medical safety data by finding and removing duplicate patient reports so each adverse event case is only counted once instead of multiple times.

In [ ]:
import pandas as pd
import numpy as np
import os
import gc
import warnings
warnings.filterwarnings('ignore')

# DRUG FILE PATHS - Focus on the massive drug dataset
DRUG_INPUT_PATH = "/Users/deliciamagdaline/Desktop/Project/faers_menstrual_rag_project/notebooks/faers_drugs_combined_2020_2024.csv"
DRUG_OUTPUT_PATH = "/Users/deliciamagdaline/Desktop/Project/faers_menstrual_rag_project/data/processed_cleaned/faers_drugs_deduplicated.csv"

def analyze_drug_file_structure():
    """
    Analyze the massive 9GB drug file structure
    """
    print("🔍 ANALYZING 9GB DRUG FILE STRUCTURE")
    print("=" * 60)
    
    if not os.path.exists(DRUG_INPUT_PATH):
        print(f"❌ Drug file not found: {DRUG_INPUT_PATH}")
        return None, None
    
    # Check file size
    file_size_gb = os.path.getsize(DRUG_INPUT_PATH) / (1024**3)
    print(f"📁 File size: {file_size_gb:.2f} GB")
    
    try:
        # Load small sample to understand structure
        print(f"📊 Loading sample to check structure...")
        sample_df = pd.read_csv(DRUG_INPUT_PATH, nrows=5000, low_memory=False)
        
        print(f"  Sample: {sample_df.shape[0]:,} rows, {sample_df.shape[1]} columns")
        print(f"  Columns: {list(sample_df.columns)}")
        
        # Find deduplication columns
        caseid_col = None
        caseversion_col = None
        
        # Look for case ID variations
        caseid_candidates = ['caseid', 'case_id', 'primaryid', 'isr']
        for col in sample_df.columns:
            if any(candidate.lower() in col.lower() for candidate in caseid_candidates):
                caseid_col = col
                break
        
        # Look for version variations  
        version_candidates = ['caseversion', 'case_version', 'version']
        for col in sample_df.columns:
            if any(candidate.lower() in col.lower() for candidate in version_candidates):
                caseversion_col = col
                break
        
        print(f"\n🎯 DEDUPLICATION COLUMNS:")
        print(f"  Case ID column: {caseid_col}")
        print(f"  Case version column: {caseversion_col}")
        
        if caseid_col and caseversion_col:
            # Show sample data
            print(f"\n📋 Sample deduplication data:")
            sample_data = sample_df[[caseid_col, caseversion_col]].head(10)
            print(sample_data)
            
            # Show version distribution
            version_counts = sample_df[caseversion_col].value_counts().head(10)
            print(f"\n📈 Version distribution (sample):")
            for version, count in version_counts.items():
                print(f"    Version {version}: {count:,} records")
        
        return caseid_col, caseversion_col
        
    except Exception as e:
        print(f"❌ Error analyzing file: {e}")
        return None, None

def estimate_drug_duplicates(caseid_col, caseversion_col, sample_size=200000):
    """
    Estimate duplicate rate in the massive drug file
    """
    print(f"\n📊 ESTIMATING DUPLICATES IN 9GB DRUG FILE")
    print("=" * 60)
    print(f"Analyzing sample of {sample_size:,} rows...")
    
    try:
        # Load larger sample for better estimate
        sample_df = pd.read_csv(DRUG_INPUT_PATH, nrows=sample_size, low_memory=False)
        
        total_rows = len(sample_df)
        unique_cases = sample_df[caseid_col].nunique()
        
        # Convert version to numeric for analysis
        sample_df[caseversion_col] = pd.to_numeric(sample_df[caseversion_col], errors='coerce')
        
        # Count cases with multiple versions
        case_counts = sample_df.groupby(caseid_col).size()
        cases_with_multiple_versions = (case_counts > 1).sum()
        
        # Find max versions per case
        max_versions = sample_df.groupby(caseid_col)[caseversion_col].max()
        avg_max_version = max_versions.mean()
        
        duplicate_rate = (total_rows - unique_cases) / total_rows * 100
        
        print(f"📈 DUPLICATE ANALYSIS RESULTS:")
        print(f"  Sample size: {total_rows:,} rows")
        print(f"  Unique cases: {unique_cases:,}")
        print(f"  Total duplicates: {total_rows - unique_cases:,}")
        print(f"  Duplicate rate: {duplicate_rate:.1f}%")
        print(f"  Cases with multiple versions: {cases_with_multiple_versions:,}")
        print(f"  Average max version: {avg_max_version:.1f}")
        
        # Estimate final file size
        estimated_final_rows = unique_cases
        size_reduction = duplicate_rate
        
        print(f"\n🎯 PROJECTED RESULTS:")
        print(f"  Estimated final rows: {estimated_final_rows:,}")
        print(f"  Estimated size reduction: {size_reduction:.1f}%")
        
        return duplicate_rate, unique_cases
        
    except Exception as e:
        print(f"❌ Error estimating duplicates: {e}")
        return 0, 0

def deduplicate_massive_drug_file(caseid_col, caseversion_col, chunk_size=25000):
    """
    Deduplicate the massive 9GB drug file using optimized chunked processing
    """
    print(f"\n🚀 DEDUPLICATING MASSIVE 9GB DRUG FILE")
    print("=" * 60)
    print(f"Strategy: Keep latest {caseversion_col} for each {caseid_col}")
    print(f"Chunk size: {chunk_size:,} rows (optimized for 9GB file)")
    
    try:
        # PHASE 1: Scan entire file to find latest version per case
        print(f"\n📋 PHASE 1: Scanning 9GB file to find latest versions...")
        print("This may take 15-30 minutes for 9GB file...")
        
        latest_versions = {}
        total_rows_scanned = 0
        chunk_count = 0
        
        # Use smaller chunks for better memory management with 9GB file
        chunk_iterator = pd.read_csv(DRUG_INPUT_PATH, chunksize=chunk_size, low_memory=False)
        
        for chunk in chunk_iterator:
            chunk_count += 1
            
            # Convert version to numeric
            chunk[caseversion_col] = pd.to_numeric(chunk[caseversion_col], errors='coerce')
            
            # Find max version per case in this chunk
            chunk_latest = chunk.groupby(caseid_col)[caseversion_col].max()
            
            # Update global latest versions
            for caseid, version in chunk_latest.items():
                if pd.notna(version):
                    if caseid not in latest_versions or version > latest_versions[caseid]:
                        latest_versions[caseid] = version
            
            total_rows_scanned += len(chunk)
            
            # Progress update every 50 chunks
            if chunk_count % 50 == 0:
                gb_processed = (total_rows_scanned * 4) / (1024**3)  # Rough estimate
                print(f"    Processed {chunk_count} chunks, ~{gb_processed:.1f}GB, {total_rows_scanned:,} rows...")
                print(f"    Found {len(latest_versions):,} unique cases so far...")
            
            # Clear chunk memory aggressively
            del chunk
            gc.collect()
        
        print(f"✅ Phase 1 complete!")
        print(f"  Total rows scanned: {total_rows_scanned:,}")
        print(f"  Unique cases found: {len(latest_versions):,}")
        print(f"  Latest versions identified for all cases")
        
        # PHASE 2: Filter and save only latest versions
        print(f"\n💾 PHASE 2: Filtering and saving latest versions...")
        print("This may take another 15-30 minutes...")
        
        # Create output directory
        os.makedirs(os.path.dirname(DRUG_OUTPUT_PATH), exist_ok=True)
        
        # Save header
        header_df = pd.read_csv(DRUG_INPUT_PATH, nrows=0)
        header_df.to_csv(DRUG_OUTPUT_PATH, index=False)
        
        # Process file again, keeping only latest versions
        kept_rows = 0
        total_processed = 0
        chunk_count = 0
        
        chunk_iterator = pd.read_csv(DRUG_INPUT_PATH, chunksize=chunk_size, low_memory=False)
        
        for chunk in chunk_iterator:
            chunk_count += 1
            chunk[caseversion_col] = pd.to_numeric(chunk[caseversion_col], errors='coerce')
            
            # Filter to keep only latest versions
            def is_latest_version(row):
                caseid = row[caseid_col]
                version = row[caseversion_col]
                
                if pd.isna(version) or caseid not in latest_versions:
                    return False
                
                return version == latest_versions[caseid]
            
            # Apply filter efficiently
            mask = chunk.apply(is_latest_version, axis=1)
            filtered_chunk = chunk[mask].copy()
            
            # Save filtered chunk
            if not filtered_chunk.empty:
                filtered_chunk.to_csv(DRUG_OUTPUT_PATH, mode='a', header=False, index=False)
                kept_rows += len(filtered_chunk)
            
            total_processed += len(chunk)
            
            # Progress update every 50 chunks
            if chunk_count % 50 == 0:
                gb_processed = (total_processed * 4) / (1024**3)
                print(f"    Processed {chunk_count} chunks, ~{gb_processed:.1f}GB")
                print(f"    Kept {kept_rows:,}/{total_processed:,} rows so far...")
            
            # Aggressive memory cleanup
            del chunk, filtered_chunk, mask
            gc.collect()
        
        # Final statistics
        removed_rows = total_processed - kept_rows
        removal_rate = (removed_rows / total_processed) * 100
        
        # Calculate file size reduction
        output_size_gb = os.path.getsize(DRUG_OUTPUT_PATH) / (1024**3) if os.path.exists(DRUG_OUTPUT_PATH) else 0
        input_size_gb = os.path.getsize(DRUG_INPUT_PATH) / (1024**3)
        size_reduction = ((input_size_gb - output_size_gb) / input_size_gb) * 100
        
        print(f"\n🎉 MASSIVE FILE DEDUPLICATION COMPLETE!")
        print("=" * 60)
        print(f"📊 FINAL STATISTICS:")
        print(f"  Input file size: {input_size_gb:.2f} GB")
        print(f"  Output file size: {output_size_gb:.2f} GB")
        print(f"  Size reduction: {size_reduction:.1f}%")
        print(f"  Total input rows: {total_processed:,}")
        print(f"  Final output rows: {kept_rows:,}")
        print(f"  Removed rows: {removed_rows:,}")
        print(f"  Row reduction: {removal_rate:.1f}%")
        print(f"  Output file: {DRUG_OUTPUT_PATH}")
        
        return {
            'input_rows': total_processed,
            'output_rows': kept_rows,
            'removed_rows': removed_rows,
            'removal_rate': removal_rate,
            'input_size_gb': input_size_gb,
            'output_size_gb': output_size_gb,
            'size_reduction': size_reduction
        }
        
    except Exception as e:
        print(f"❌ Error during massive file deduplication: {e}")
        return None

def run_drug_deduplication():
    """
    Complete drug file deduplication workflow
    """
    print("💊 MASSIVE DRUG FILE DEDUPLICATION WORKFLOW")
    print("=" * 70)
    print("Optimized for 9GB FAERS drug dataset")
    
    # Step 1: Analyze file structure
    caseid_col, caseversion_col = analyze_drug_file_structure()
    
    if not caseid_col or not caseversion_col:
        print("❌ Cannot proceed - missing required columns")
        return None
    
    # Step 2: Estimate duplicates
    duplicate_rate, unique_cases = estimate_drug_duplicates(caseid_col, caseversion_col)
    
    # Step 3: Ask for confirmation before processing 9GB file
    print(f"\n⚠️  READY TO PROCESS 9GB FILE")
    print(f"This will take 30-60 minutes and use significant processing power.")
    print(f"Estimated duplicate removal: {duplicate_rate:.1f}%")
    print(f"Proceed with deduplication? Uncomment the line below:")
    print(f"# result = deduplicate_massive_drug_file('{caseid_col}', '{caseversion_col}')")
    
    return caseid_col, caseversion_col

# 🚀 MAIN EXECUTION
if __name__ == "__main__":
    print("💊 MASSIVE DRUG FILE DEDUPLICATION TOOLKIT")
    print("=" * 70)
    print("Specialized for 9GB FAERS drug dataset")
    print("FDA Standard: Keep latest CASEVERSION per CASEID")
    
    print(f"\n🎯 OPTIMIZATIONS FOR 9GB FILE:")
    print("✅ Small chunk sizes (25K rows) for memory efficiency")
    print("✅ Two-phase processing (scan → filter)")
    print("✅ Aggressive memory cleanup between chunks")
    print("✅ Progress tracking for long-running process")
    print("✅ File size reduction statistics")
    
    print(f"\n⏱️  EXPECTED TIME:")
    print("📋 Phase 1 (scan): 15-30 minutes")
    print("💾 Phase 2 (filter): 15-30 minutes") 
    print("🎯 Total: 30-60 minutes for 9GB file")
    
    print(f"\n🚀 TO START:")
    print("# caseid_col, caseversion_col = run_drug_deduplication()")
    
    # Uncomment to run analysis:
    # caseid_col, caseversion_col = run_drug_deduplication()
    
    # After analysis, uncomment to run full deduplication:
    # result = deduplicate_massive_drug_file(caseid_col, caseversion_col)

####  Standardize Your Data Demo DF"
This is the most critical step.

Dates: Convert all date fields (EVENT_DT, INIT_FDA_DT) to a single, consistent format (e.g., YYYY-MM-DD). Be aware that dates can be incomplete.

Age: Standardize PATIENT_AGE into a single unit (years). FAERS reports age in various units (decades, years, months, days). You'll need to write a script to convert everything to years.

Sex: Clean the PATIENT_SEX field to a uniform format (e.g., 'M', 'F', 'U' for Unknown).

In [1]:
import pandas as pd
import numpy as np
import re
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# FILE PATHS
DEMO_FILE_PATH = "/Users/deliciamagdaline/Desktop/Project/faers_menstrual_rag_project/data/processed/faers_demographics_combined_2020_2024.csv"
OUTPUT_FILE_PATH = "/Users/deliciamagdaline/Desktop/Project/faers_menstrual_rag_project/data/processed_cleaned/faers_demographics_cleaned.csv"

# ESSENTIAL COLUMNS TO CLEAN (but we'll keep ALL columns in the file)
ESSENTIAL_COLUMNS_TO_CLEAN = [
    'primaryid',        # Key identifier
    'caseid',          # Alternative key
    'event_dt',        # Main event date
    'init_fda_dt',     # FDA initial date
    'rept_dt',         # Report date
    'age',             # Age value
    'age_cod',         # Age unit
    'sex',             # Sex/gender
    'reporter_country', # Country (useful for analysis)
    'year',            # Year (useful for filtering)
    'quarter'          # Quarter (useful for filtering)
]

def load_all_columns():
    """
    Load ALL columns but focus cleaning on essential ones only
    """
    print("📊 Loading ALL COLUMNS (cleaning only essential ones for speed)")
    print(f"Will clean these essential columns: {ESSENTIAL_COLUMNS_TO_CLEAN}")
    
    try:
        # Load ALL columns
        demo_df = pd.read_csv(DEMO_FILE_PATH, low_memory=False)
        print(f"✅ Loaded: {demo_df.shape[0]:,} rows, {demo_df.shape[1]} columns")
        
        # Check which essential columns are available
        available_essential = [col for col in ESSENTIAL_COLUMNS_TO_CLEAN if col in demo_df.columns]
        missing_essential = [col for col in ESSENTIAL_COLUMNS_TO_CLEAN if col not in demo_df.columns]
        
        print(f"📋 Available essential columns: {available_essential}")
        if missing_essential:
            print(f"⚠️  Missing essential columns: {missing_essential}")
        
        print(f"📋 ALL columns in file:")
        for i, col in enumerate(demo_df.columns, 1):
            status = "🎯" if col in ESSENTIAL_COLUMNS_TO_CLEAN else "📁"
            print(f"  {i:2d}. {status} {col}")
        
        return demo_df, available_essential
    
    except Exception as e:
        print(f"❌ Error loading file: {e}")
        return None, []

def check_data_quality_essential_only(demo_df, available_essential):
    """
    Check data quality focusing ONLY on essential columns
    Adds 'data_quality_flag' column based on essential data
    """
    print("🔍 CHECKING DATA QUALITY (Essential Columns Only)")
    
    demo_clean = demo_df.copy()
    
    # Define critical columns that should not be empty (from essential list)
    critical_columns = ['primaryid'] if 'primaryid' in available_essential else []
    
    # Define important columns (from available essential)
    important_columns = [col for col in ['event_dt', 'age', 'sex'] if col in available_essential]
    
    # Define useful columns (from available essential)
    useful_columns = [col for col in ['init_fda_dt', 'rept_dt', 'age_cod', 'reporter_country'] if col in available_essential]
    
    print(f"Quality assessment using:")
    print(f"  Critical: {critical_columns}")
    print(f"  Important: {important_columns}")
    print(f"  Useful: {useful_columns}")
    
    # Count missing values for each row (only essential columns)
    def assess_row_quality(row):
        # Critical: If missing primaryid, definitely remove
        if critical_columns and (pd.isna(row.get(critical_columns[0])) or row.get(critical_columns[0]) == ''):
            return 'REMOVE_NO_ID'
        
        # Count missing important fields
        important_missing = 0
        for col in important_columns:
            if pd.isna(row[col]) or row[col] == '' or row[col] == 'nan':
                important_missing += 1
        
        # Count missing useful fields  
        useful_missing = 0
        for col in useful_columns:
            if pd.isna(row[col]) or row[col] == '' or row[col] == 'nan':
                useful_missing += 1
        
        # Decision logic
        if important_missing >= len(important_columns):  # Missing all important fields
            return 'REMOVE_TOO_EMPTY'
        elif important_missing >= len(important_columns) - 1:  # Missing most important fields
            return 'CONSIDER_REMOVE'
        elif important_missing >= 1:  # Missing some important fields
            return 'KEEP_WITH_CAUTION'
        else:
            return 'KEEP_GOOD_QUALITY'
    
    # Apply quality assessment
    demo_clean['data_quality_flag'] = demo_clean.apply(assess_row_quality, axis=1)
    
    # Add numeric quality score (0-100, higher = better)
    def calculate_quality_score(row):
        if not (critical_columns or important_columns or useful_columns):
            return 100  # If no essential columns to check, assume good quality
            
        total_possible = len(critical_columns) * 3 + len(important_columns) * 2 + len(useful_columns) * 1
        if total_possible == 0:
            return 100
            
        filled_fields = 0
        
        # Critical fields (weight = 3)
        for col in critical_columns:
            if pd.notna(row[col]) and row[col] != '' and row[col] != 'nan':
                filled_fields += 3
        
        # Important fields (weight = 2)  
        for col in important_columns:
            if pd.notna(row[col]) and row[col] != '' and row[col] != 'nan':
                filled_fields += 2
        
        # Useful fields (weight = 1)
        for col in useful_columns:
            if pd.notna(row[col]) and row[col] != '' and row[col] != 'nan':
                filled_fields += 1
        
        return round((filled_fields / total_possible) * 100, 1)
    
    demo_clean['data_quality_score'] = demo_clean.apply(calculate_quality_score, axis=1)
    
    # Report quality statistics
    quality_counts = demo_clean['data_quality_flag'].value_counts()
    total_rows = len(demo_clean)
    
    print(f"\nDATA QUALITY ASSESSMENT:")
    print(f"  Total rows: {total_rows:,}")
    
    for flag, count in quality_counts.items():
        pct = (count / total_rows) * 100
        status = "🔴" if "REMOVE" in flag else "🟡" if "CAUTION" in flag else "🟢"
        print(f"  {status} {flag}: {count:,} ({pct:.1f}%)")
    
    # Summary recommendations
    remove_rows = quality_counts.get('REMOVE_NO_ID', 0) + quality_counts.get('REMOVE_TOO_EMPTY', 0)
    caution_rows = quality_counts.get('CONSIDER_REMOVE', 0) + quality_counts.get('KEEP_WITH_CAUTION', 0)
    keep_rows = quality_counts.get('KEEP_GOOD_QUALITY', 0)
    
    print(f"\nRECOMMENDATIONS:")
    print(f"  🔴 Definitely remove: {remove_rows:,} rows ({(remove_rows/total_rows)*100:.1f}%)")
    print(f"  🟡 Review carefully: {caution_rows:,} rows ({(caution_rows/total_rows)*100:.1f}%)")
    print(f"  🟢 Keep (good quality): {keep_rows:,} rows ({(keep_rows/total_rows)*100:.1f}%)")
    
    return demo_clean

def standardize_date_string(date_str):
    """
    Convert various date formats to YYYY-MM-DD
    Handles incomplete dates by padding with defaults
    """
    if not date_str or date_str == '' or pd.isna(date_str):
        return None
    
    # Remove any non-digit characters except hyphens and slashes
    date_str = re.sub(r'[^\d\-/]', '', str(date_str))
    
    if not date_str:
        return None
    
    # Handle different date formats
    patterns = [
        # YYYYMMDD
        (r'^(\d{4})(\d{2})(\d{2})$', r'\1-\2-\3'),
        # YYYY-MM-DD or YYYY/MM/DD
        (r'^(\d{4})[-/](\d{1,2})[-/](\d{1,2})$', r'\1-\2-\3'),
        # MM/DD/YYYY or MM-DD-YYYY
        (r'^(\d{1,2})[-/](\d{1,2})[-/](\d{4})$', r'\3-\1-\2'),
        # DD/MM/YYYY or DD-MM-YYYY (assuming day first if day > 12)
        (r'^(\d{1,2})[-/](\d{1,2})[-/](\d{4})$', r'\3-\2-\1'),
        # YYYY-MM (incomplete - pad day with 01)
        (r'^(\d{4})[-/](\d{1,2})$', r'\1-\2-01'),
        # YYYY only (incomplete - pad with 01-01)
        (r'^(\d{4})$', r'\1-01-01'),
    ]
    
    for pattern, replacement in patterns:
        if re.match(pattern, date_str):
            result = re.sub(pattern, replacement, date_str)
            # Ensure two-digit month and day
            parts = result.split('-')
            if len(parts) == 3:
                year, month, day = parts
                try:
                    # Validate and format
                    month = f"{int(month):02d}"
                    day = f"{int(day):02d}"
                    
                    # Basic validation
                    if 1 <= int(month) <= 12 and 1 <= int(day) <= 31:
                        return f"{year}-{month}-{day}"
                except ValueError:
                    continue
    
    return None

def clean_essential_dates_only(demo_df, available_essential):
    """
    Clean ONLY the essential date columns
    """
    # Only clean essential date columns that exist
    essential_dates = ['event_dt', 'init_fda_dt', 'rept_dt']
    priority_dates = [col for col in essential_dates if col in available_essential]
    
    print(f"🗓️  Cleaning ESSENTIAL date columns only: {priority_dates}")
    
    demo_clean = demo_df.copy()
    
    for col in priority_dates:
        print(f"\nProcessing {col}...")
        original_count = demo_clean[col].notna().sum()
        
        # Convert to string and handle missing values
        demo_clean[col] = demo_clean[col].astype(str).replace('nan', '')
        
        # Clean the date strings
        demo_clean[f'{col}_cleaned'] = demo_clean[col].apply(standardize_date_string)
        
        # Convert to datetime
        demo_clean[f'{col}_standardized'] = pd.to_datetime(
            demo_clean[f'{col}_cleaned'], 
            format='%Y-%m-%d', 
            errors='coerce'
        )
        
        # Count successful conversions
        clean_count = demo_clean[f'{col}_standardized'].notna().sum()
        success_rate = (clean_count / original_count * 100) if original_count > 0 else 0
        
        print(f"  Original non-null: {original_count:,}")
        print(f"  Successfully cleaned: {clean_count:,}")
        print(f"  Success rate: {success_rate:.1f}%")
    
    return demo_clean

def clean_essential_age_only(demo_df, available_essential):
    """
    Clean ONLY essential age columns
    """
    print(f"👶 Cleaning essential age data")
    
    demo_clean = demo_df.copy()
    
    # Check if essential age columns exist
    has_age = 'age' in available_essential
    has_age_cod = 'age_cod' in available_essential
    
    if not has_age:
        print(f"⚠️  Essential age column not found")
        return demo_clean
    
    print(f"Found age columns: age={has_age}, age_cod={has_age_cod}")
    
    # Convert age to numeric
    demo_clean['age'] = pd.to_numeric(demo_clean['age'], errors='coerce')
    
    if has_age_cod:
        # Standardize age unit codes
        demo_clean['age_cod'] = demo_clean['age_cod'].astype(str).str.upper()
        
        # Simple age conversion - most common units only
        age_conversions = {
            'YR': 1.0,      # Years
            'MON': 1/12,    # Months 
            'DY': 1/365,    # Days
            'DEC': 10.0,    # Decades
            'WK': 1/52,     # Weeks
        }
        
        # Vectorized conversion for speed
        def convert_age_to_years(row):
            age_val = row['age']
            age_unit = row['age_cod']
            
            if pd.isna(age_val) or pd.isna(age_unit):
                return None
            
            if age_unit in age_conversions:
                converted_age = age_val * age_conversions[age_unit]
                if 0 <= converted_age <= 120:
                    return round(converted_age, 2)
            
            return None
        
        demo_clean['age_in_years'] = demo_clean.apply(convert_age_to_years, axis=1)
    else:
        # If no age_cod, assume age is already in years
        demo_clean['age_in_years'] = demo_clean['age'].apply(
            lambda x: round(x, 2) if pd.notna(x) and 0 <= x <= 120 else None
        )
    
    # Simple age groups
    demo_clean['age_group'] = pd.cut(
        demo_clean['age_in_years'], 
        bins=[0, 18, 35, 65, 120], 
        labels=['Child', 'Young Adult', 'Middle-aged', 'Senior'],
        include_lowest=True
    )
    
    # Quick stats
    original_count = demo_clean['age'].notna().sum()
    converted_count = demo_clean['age_in_years'].notna().sum()
    success_rate = (converted_count / original_count * 100) if original_count > 0 else 0
    
    print(f"  Age conversion success rate: {success_rate:.1f}%")
    
    return demo_clean

def clean_essential_sex_only(demo_df, available_essential):
    """
    Clean ONLY essential sex column
    """
    print(f"⚥ Cleaning essential sex data")
    
    demo_clean = demo_df.copy()
    
    if 'sex' not in available_essential:
        print(f"⚠️  Essential sex column not found")
        return demo_clean
    
    # Simple sex standardization
    demo_clean['sex'] = demo_clean['sex'].astype(str).str.upper().str.strip()
    
    # Quick mapping
    sex_map = {'M': 'M', 'F': 'F'}
    demo_clean['sex_standardized'] = demo_clean['sex'].map(sex_map).fillna('U')
    
    # Quick stats
    sex_counts = demo_clean['sex_standardized'].value_counts()
    total = len(demo_clean)
    
    print(f"  Sex distribution:")
    for sex, count in sex_counts.items():
        pct = (count / total) * 100
        print(f"    {sex}: {pct:.1f}%")
    
    return demo_clean

def save_large_csv_chunked(df, output_path, chunk_size=100000):
    """
    Save large DataFrame to CSV in chunks to avoid memory issues and KeyboardInterrupt
    """
    print(f"💾 Saving {len(df):,} rows in chunks of {chunk_size:,} to avoid crashes...")
    
    try:
        # Save header first
        df.head(0).to_csv(output_path, index=False)
        
        # Save in chunks
        total_chunks = len(df) // chunk_size + (1 if len(df) % chunk_size > 0 else 0)
        
        for i in range(0, len(df), chunk_size):
            chunk_num = i // chunk_size + 1
            chunk = df.iloc[i:i+chunk_size]
            chunk.to_csv(output_path, mode='a', header=False, index=False)
            print(f"  ✅ Saved chunk {chunk_num}/{total_chunks}: rows {i:,} to {min(i+chunk_size, len(df)):,}")
        
        print(f"✅ Successfully saved all {len(df):,} rows to {output_path}")
        return True
        
    except Exception as e:
        print(f"❌ Error saving file: {e}")
        return False

def clean_demographics_all_columns():
    """
    Clean demographics with ALL columns preserved, but only standardize essential ones
    """
    print("🚀 FAERS DEMOGRAPHICS - KEEP ALL COLUMNS, CLEAN ESSENTIALS")
    print("=" * 70)
    print("Strategy: Load everything, clean only important columns!")
    
    # Step 1: Load ALL columns
    demo_df, available_essential = load_all_columns()
    if demo_df is None:
        return None
    
    original_rows = len(demo_df)
    original_cols = len(demo_df.columns)
    
    # Step 2: Check data quality (essential columns only)
    print(f"\n" + "=" * 50)
    demo_clean = check_data_quality_essential_only(demo_df, available_essential)
    
    # Step 3: REMOVE poor quality rows automatically
    print(f"\n" + "=" * 50)
    print("🗑️  REMOVING POOR QUALITY ROWS...")
    
    rows_to_remove = demo_clean['data_quality_flag'].isin(['REMOVE_NO_ID', 'REMOVE_TOO_EMPTY'])
    removed_count = rows_to_remove.sum()
    
    demo_clean = demo_clean[~rows_to_remove].copy()
    
    print(f"  Removed {removed_count:,} poor quality rows")
    print(f"  Remaining: {len(demo_clean):,} rows ({((len(demo_clean)/original_rows)*100):.1f}% of original)")
    
    # Step 4: Clean essential dates only
    print(f"\n" + "=" * 50)
    demo_clean = clean_essential_dates_only(demo_clean, available_essential)
    
    # Step 5: Clean essential age only
    print(f"\n" + "=" * 50)
    demo_clean = clean_essential_age_only(demo_clean, available_essential)
    
    # Step 6: Clean essential sex only
    print(f"\n" + "=" * 50)
    demo_clean = clean_essential_sex_only(demo_clean, available_essential)
    
    # Step 7: Save ALL data with cleaned essentials
    print(f"\n" + "=" * 50)
    print("💾 Saving ALL COLUMNS with cleaned essentials...")
    
    # Always use chunked saving to prevent crashes
    save_success = save_large_csv_chunked(demo_clean, OUTPUT_FILE_PATH, chunk_size=50000)
    
    if save_success:
        print(f"\n✅ COMPLETE SUCCESS!")
    else:
        print(f"\n❌ Error during saving - but cleaning completed successfully")
    
    # Final summary
    final_rows = len(demo_clean)
    final_cols = len(demo_clean.columns)
    removal_rate = (removed_count / original_rows) * 100
    
    print(f"\nFINAL SUMMARY:")
    print(f"  Original records: {original_rows:,}")
    print(f"  Original columns: {original_cols}")
    print(f"  🗑️  Removed (poor quality): {removed_count:,} ({removal_rate:.1f}%)")
    print(f"  ✅ Final clean records: {final_rows:,}")
    print(f"  ✅ Final columns: {final_cols} (all preserved + new clean columns)")
    
    # Show cleaning results for available essential columns
    if 'event_dt_standardized' in demo_clean.columns:
        print(f"  Valid event dates: {demo_clean['event_dt_standardized'].notna().sum():,}")
    if 'age_in_years' in demo_clean.columns:
        print(f"  Valid ages: {demo_clean['age_in_years'].notna().sum():,}")
    if 'sex_standardized' in demo_clean.columns:
        print(f"  Valid sex (M/F): {(demo_clean['sex_standardized'].isin(['M', 'F'])).sum():,}")
    
    print(f"  File saved to: {OUTPUT_FILE_PATH}")
    print(f"\n💡 Added columns:")
    print(f"     - data_quality_flag & data_quality_score")
    print(f"     - *_standardized columns for cleaned essential data")
    print(f"     - age_in_years & age_group")
    
    return demo_clean

def preview_all_columns():
    """
    Quick preview of ALL columns and which will be cleaned
    """
    print("🔍 PREVIEW - ALL COLUMNS + CLEANING PLAN")
    print("=" * 60)
    
    # Load just first 1000 rows to preview
    try:
        sample_df = pd.read_csv(DEMO_FILE_PATH, nrows=1000)
        print(f"Sample: {sample_df.shape[0]} rows, {sample_df.shape[1]} columns")
        
        print(f"\nALL COLUMNS (🎯 = will be cleaned):")
        for i, col in enumerate(sample_df.columns, 1):
            status = "🎯" if col in ESSENTIAL_COLUMNS_TO_CLEAN else "📁"
            print(f"  {i:2d}. {status} {col}")
        
        print(f"\n📋 CLEANING SUMMARY:")
        essential_found = [col for col in ESSENTIAL_COLUMNS_TO_CLEAN if col in sample_df.columns]
        essential_missing = [col for col in ESSENTIAL_COLUMNS_TO_CLEAN if col not in sample_df.columns]
        
        print(f"  Will clean: {essential_found}")
        if essential_missing:
            print(f"  Missing (won't clean): {essential_missing}")
        
        print(f"\nSample data:")
        print(sample_df[essential_found[:5]].head(3))
        
        return True
    
    except Exception as e:
        print(f"Error loading preview: {e}")
        return False

# 🚀 MAIN EXECUTION
if __name__ == "__main__":
    print("🎯 KEEP ALL COLUMNS + CLEAN ESSENTIALS STRATEGY")
    print("=" * 70)
    print("Best of both worlds: Complete data + Fast cleaning!")
    
    # Option 1: Preview first
    print("\n1️⃣ PREVIEW (see all columns + cleaning plan):")
    print("# preview_all_columns()")
    
    # Option 2: Run complete cleaning 
    print("\n2️⃣ RUN COMPLETE CLEANING:")
    print("# demo_cleaned = clean_demographics_all_columns()")
    
    print(f"\n🎯 STRATEGY BENEFITS:")
    print("✅ ALL original columns preserved")
    print("✅ Only essential columns cleaned (faster)")
    print("✅ Quality assessment on essential data")
    print("✅ Automatic removal of junk rows")
    print("✅ Chunked saving (no crashes)")
    print("✅ New standardized columns added")
    
    print(f"\n📊 RESULT:")
    print("Complete dataset with all original data PLUS cleaned essential columns")
    
    # Uncomment to run:
    preview_all_columns()
    demo_cleaned = clean_demographics_all_columns()

/Users/deliciamagdaline/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


🎯 KEEP ALL COLUMNS + CLEAN ESSENTIALS STRATEGY
Best of both worlds: Complete data + Fast cleaning!

1️⃣ PREVIEW (see all columns + cleaning plan):
# preview_all_columns()

2️⃣ RUN COMPLETE CLEANING:
# demo_cleaned = clean_demographics_all_columns()

🎯 STRATEGY BENEFITS:
✅ ALL original columns preserved
✅ Only essential columns cleaned (faster)
✅ Quality assessment on essential data
✅ Automatic removal of junk rows
✅ Chunked saving (no crashes)
✅ New standardized columns added

📊 RESULT:
Complete dataset with all original data PLUS cleaned essential columns
🔍 PREVIEW - ALL COLUMNS + CLEANING PLAN
Sample: 1000 rows, 28 columns

ALL COLUMNS (🎯 = will be cleaned):
   1. 🎯 primaryid
   2. 🎯 caseid
   3. 📁 caseversion
   4. 📁 i_f_code
   5. 🎯 event_dt
   6. 📁 mfr_dt
   7. 🎯 init_fda_dt
   8. 📁 fda_dt
   9. 📁 rept_cod
  10. 📁 auth_num
  11. 📁 mfr_num
  12. 📁 mfr_sndr
  13. 📁 lit_ref
  14. 🎯 age
  15. 🎯 age_cod
  16. 📁 age_grp
  17. 🎯 sex
  18. 📁 e_sub
  19. 📁 wt
  20. 📁 wt_cod
  21. 🎯 rept_dt

 This code removes duplicate drug records from your massive 9GB file by keeping only the newest version of each case, shrinking the file size and giving you cleaner data.

This still removes duplicate drug/patient records by keeping only the newest version of each case, but now it won't crash when your files are too massive to read normally.

In [4]:
import pandas as pd
import numpy as np
import os
import gc
import warnings
warnings.filterwarnings('ignore')

# FILE PATHS - Using your cleaned demographics file
DEMO_CLEANED_PATH = "/Users/deliciamagdaline/Desktop/Project/faers_menstrual_rag_project/data/processed_cleaned/faers_demographics_cleaned.csv"
DEMO_DEDUPE_PATH = "/Users/deliciamagdaline/Desktop/Project/faers_menstrual_rag_project/data/processed_cleaned/faers_demographics_deduplicated.csv"

DRUG_INPUT_PATH = "/Users/deliciamagdaline/Desktop/Project/faers_menstrual_rag_project/data/processed/faers_drugs_combined_2020_2024.csv"
DRUG_DEDUPE_PATH = "/Users/deliciamagdaline/Desktop/Project/faers_menstrual_rag_project/data/processed_cleaned/faers_drugs_deduplicated.csv"

def quick_file_check(file_path):
    """
    Quick file accessibility and size check
    """
    print(f"🔍 Quick check: {os.path.basename(file_path)}")
    
    if not os.path.exists(file_path):
        print(f"❌ File not found")
        return False
    
    try:
        file_size_gb = os.path.getsize(file_path) / (1024**3)
        print(f"✅ File exists: {file_size_gb:.2f} GB")
        return True
    except Exception as e:
        print(f"❌ File access error: {e}")
        return False

def safe_analyze_file(file_path, file_type, tiny_sample=100):
    """
    TIMEOUT-RESISTANT file analysis with very small samples
    """
    print(f"\n🔍 SAFE ANALYSIS: {file_type.upper()}")
    print("=" * 50)
    
    if not quick_file_check(file_path):
        return None, None
    
    try:
        # STEP 1: Try to read just header first
        print(f"📋 Reading header...")
        header_df = pd.read_csv(file_path, nrows=0, low_memory=False)
        columns = list(header_df.columns)
        print(f"✅ Found {len(columns)} columns")
        
        # STEP 2: Try tiny sample
        print(f"📊 Reading tiny sample ({tiny_sample} rows)...")
        tiny_df = pd.read_csv(file_path, nrows=tiny_sample, low_memory=False)
        print(f"✅ Sample loaded: {tiny_df.shape}")
        
        # STEP 3: Find deduplication columns
        print(f"🎯 Looking for deduplication columns...")
        
        caseid_col = None
        caseversion_col = None
        
        # Look for case ID (prioritize caseid over primaryid)
        caseid_candidates = ['caseid', 'primaryid', 'case_id', 'isr']
        for candidate in caseid_candidates:
            if candidate in columns:
                caseid_col = candidate
                print(f"✅ Found case ID: {caseid_col}")
                break
        
        # Look for version
        version_candidates = ['caseversion', 'version', 'case_version']
        for candidate in version_candidates:
            if candidate in columns:
                caseversion_col = candidate
                print(f"✅ Found version: {caseversion_col}")
                break
        
        if caseid_col and caseversion_col:
            # Show tiny sample of key columns
            print(f"\n📋 Sample data:")
            key_sample = tiny_df[[caseid_col, caseversion_col]].head(5)
            print(key_sample)
            
            # Quick version check
            versions = tiny_df[caseversion_col].unique()
            print(f"\n📈 Versions found in sample: {sorted(versions)}")
            
        return caseid_col, caseversion_col
        
    except Exception as e:
        print(f"❌ Error in safe analysis: {e}")
        return None, None

def estimate_duplicates_safely(file_path, caseid_col, caseversion_col, sample_size=1000):
    """
    Safe duplicate estimation with very small sample
    """
    print(f"\n📊 SAFE DUPLICATE ESTIMATION")
    print(f"Using tiny sample of {sample_size} rows to avoid timeout...")
    
    try:
        # Use very small sample to avoid timeout
        sample_df = pd.read_csv(file_path, nrows=sample_size, low_memory=False)
        
        total_rows = len(sample_df)
        unique_cases = sample_df[caseid_col].nunique()
        duplicate_rate = (total_rows - unique_cases) / total_rows * 100
        
        print(f"📈 QUICK ESTIMATE (tiny sample):")
        print(f"  Sample size: {total_rows}")
        print(f"  Unique cases: {unique_cases}")
        print(f"  Estimated duplicate rate: {duplicate_rate:.1f}%")
        
        # Version analysis
        sample_df[caseversion_col] = pd.to_numeric(sample_df[caseversion_col], errors='coerce')
        version_counts = sample_df[caseversion_col].value_counts()
        print(f"  Version distribution: {dict(version_counts.head())}")
        
        return duplicate_rate
        
    except Exception as e:
        print(f"❌ Error in estimation: {e}")
        return 0

def deduplicate_small_file_fast(input_path, output_path, caseid_col, caseversion_col, file_type):
    """
    Fast in-memory deduplication for smaller files (like cleaned demographics)
    """
    print(f"\n🚀 FAST DEDUPLICATION: {file_type.upper()}")
    print("=" * 50)
    
    try:
        # Load entire file (assuming it's not too large)
        print(f"📊 Loading entire file...")
        df = pd.read_csv(input_path, low_memory=False)
        print(f"✅ Loaded: {len(df):,} rows, {len(df.columns)} columns")
        
        # Convert version to numeric for proper sorting
        print(f"🔄 Converting versions to numeric...")
        df[caseversion_col] = pd.to_numeric(df[caseversion_col], errors='coerce')
        
        # Keep only latest version per case
        print(f"🎯 Keeping latest version per case...")
        df_dedupe = df.sort_values([caseid_col, caseversion_col]).groupby(caseid_col).tail(1)
        
        # Save result
        print(f"💾 Saving deduplicated file...")
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        df_dedupe.to_csv(output_path, index=False)
        
        # Statistics
        original_rows = len(df)
        final_rows = len(df_dedupe)
        removed_rows = original_rows - final_rows
        removal_rate = (removed_rows / original_rows) * 100
        
        input_size_mb = os.path.getsize(input_path) / (1024**2)
        output_size_mb = os.path.getsize(output_path) / (1024**2)
        size_reduction = ((input_size_mb - output_size_mb) / input_size_mb) * 100
        
        print(f"\n✅ DEDUPLICATION COMPLETE!")
        print(f"📊 RESULTS:")
        print(f"  Original rows: {original_rows:,}")
        print(f"  Final rows: {final_rows:,}")
        print(f"  Removed: {removed_rows:,} ({removal_rate:.1f}%)")
        print(f"  Size: {input_size_mb:.1f} MB → {output_size_mb:.1f} MB ({size_reduction:.1f}% reduction)")
        print(f"  Output: {output_path}")
        
        return {
            'success': True,
            'original_rows': original_rows,
            'final_rows': final_rows,
            'removal_rate': removal_rate,
            'size_reduction': size_reduction
        }
        
    except Exception as e:
        print(f"❌ Error during deduplication: {e}")
        return {'success': False, 'error': str(e)}

def deduplicate_massive_file_safe(input_path, output_path, caseid_col, caseversion_col, file_type):
    """
    Safe chunked deduplication for massive files (like 9GB drug file)
    """
    print(f"\n🚀 MASSIVE FILE DEDUPLICATION: {file_type.upper()}")
    print("=" * 50)
    print("⚠️  This will take 30-60 minutes for 9GB file")
    
    # Get file size
    file_size_gb = os.path.getsize(input_path) / (1024**3)
    print(f"📁 File size: {file_size_gb:.2f} GB")
    
    # Use very small chunks to avoid timeout
    chunk_size = 10000  # Extra small for safety
    
    try:
        # PHASE 1: Find latest versions
        print(f"\n📋 PHASE 1: Finding latest versions (chunk size: {chunk_size:,})...")
        
        latest_versions = {}
        total_scanned = 0
        chunk_count = 0
        
        chunk_iterator = pd.read_csv(input_path, chunksize=chunk_size, low_memory=False)
        
        for chunk in chunk_iterator:
            chunk_count += 1
            
            # Convert version to numeric
            chunk[caseversion_col] = pd.to_numeric(chunk[caseversion_col], errors='coerce')
            
            # Find max version per case in chunk
            chunk_latest = chunk.groupby(caseid_col)[caseversion_col].max()
            
            # Update global latest versions
            for caseid, version in chunk_latest.items():
                if pd.notna(version):
                    if caseid not in latest_versions or version > latest_versions[caseid]:
                        latest_versions[caseid] = version
            
            total_scanned += len(chunk)
            
            # Progress every 100 chunks (less frequent to avoid spam)
            if chunk_count % 100 == 0:
                print(f"    {chunk_count} chunks, {total_scanned:,} rows, {len(latest_versions):,} unique cases...")
            
            # Memory cleanup
            del chunk
            gc.collect()
        
        print(f"✅ Phase 1 complete: {len(latest_versions):,} unique cases found")
        
        # PHASE 2: Filter and save
        print(f"\n💾 PHASE 2: Filtering and saving...")
        
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        
        # Save header
        header_df = pd.read_csv(input_path, nrows=0)
        header_df.to_csv(output_path, index=False)
        
        kept_rows = 0
        total_processed = 0
        chunk_count = 0
        
        chunk_iterator = pd.read_csv(input_path, chunksize=chunk_size, low_memory=False)
        
        for chunk in chunk_iterator:
            chunk_count += 1
            chunk[caseversion_col] = pd.to_numeric(chunk[caseversion_col], errors='coerce')
            
            # Filter efficiently
            mask = (
                chunk[caseid_col].isin(latest_versions.keys()) &
                chunk.apply(lambda row: (
                    pd.notna(row[caseversion_col]) and 
                    row[caseversion_col] == latest_versions.get(row[caseid_col], -1)
                ), axis=1)
            )
            
            filtered_chunk = chunk[mask]
            
            if not filtered_chunk.empty:
                filtered_chunk.to_csv(output_path, mode='a', header=False, index=False)
                kept_rows += len(filtered_chunk)
            
            total_processed += len(chunk)
            
            if chunk_count % 100 == 0:
                print(f"    {chunk_count} chunks, kept {kept_rows:,}/{total_processed:,} rows...")
            
            del chunk, filtered_chunk, mask
            gc.collect()
        
        # Final stats
        removed_rows = total_processed - kept_rows
        removal_rate = (removed_rows / total_processed) * 100
        
        output_size_gb = os.path.getsize(output_path) / (1024**3)
        size_reduction = ((file_size_gb - output_size_gb) / file_size_gb) * 100
        
        print(f"\n🎉 MASSIVE FILE DEDUPLICATION COMPLETE!")
        print(f"📊 FINAL RESULTS:")
        print(f"  Input: {total_processed:,} rows ({file_size_gb:.2f} GB)")
        print(f"  Output: {kept_rows:,} rows ({output_size_gb:.2f} GB)")
        print(f"  Removed: {removed_rows:,} ({removal_rate:.1f}%)")
        print(f"  Size reduction: {size_reduction:.1f}%")
        
        return {
            'success': True,
            'original_rows': total_processed,
            'final_rows': kept_rows,
            'removal_rate': removal_rate,
            'size_reduction': size_reduction
        }
        
    except Exception as e:
        print(f"❌ Error during massive deduplication: {e}")
        return {'success': False, 'error': str(e)}

def run_safe_deduplication():
    """
    Safe deduplication workflow that handles timeouts
    """
    print("🛡️ TIMEOUT-SAFE FAERS DEDUPLICATION")
    print("=" * 70)
    
    results = {}
    
    # STEP 1: Analyze cleaned demographics
    print(f"\n{'='*70}")
    print("STEP 1: ANALYZING CLEANED DEMOGRAPHICS")
    demo_caseid, demo_version = safe_analyze_file(DEMO_CLEANED_PATH, "cleaned demographics", tiny_sample=50)
    
    if demo_caseid and demo_version:
        # Estimate duplicates
        estimate_duplicates_safely(DEMO_CLEANED_PATH, demo_caseid, demo_version, sample_size=500)
        
        # Option to deduplicate
        print(f"\n🚀 Ready to deduplicate demographics")
        print(f"# demo_result = deduplicate_small_file_fast(DEMO_CLEANED_PATH, DEMO_DEDUPE_PATH, '{demo_caseid}', '{demo_version}', 'demographics')")
    
    # STEP 2: Analyze drug file
    print(f"\n{'='*70}")
    print("STEP 2: ANALYZING DRUG FILE") 
    drug_caseid, drug_version = safe_analyze_file(DRUG_INPUT_PATH, "drug", tiny_sample=50)
    
    if drug_caseid and drug_version:
        # Estimate duplicates
        estimate_duplicates_safely(DRUG_INPUT_PATH, drug_caseid, drug_version, sample_size=500)
        
        # Option to deduplicate
        print(f"\n🚀 Ready to deduplicate drug file")
        print(f"# drug_result = deduplicate_massive_file_safe(DRUG_INPUT_PATH, DRUG_DEDUPE_PATH, '{drug_caseid}', '{drug_version}', 'drug')")
    
    return demo_caseid, demo_version, drug_caseid, drug_version

# 🚀 MAIN EXECUTION
if __name__ == "__main__":
    print("🛡️ TIMEOUT-SAFE FAERS DEDUPLICATION")
    print("=" * 70)
    print("Handles massive files without timeout errors!")
    
    print(f"\n🔧 SAFETY FEATURES:")
    print("✅ Tiny sample sizes (50-1000 rows) for analysis")
    print("✅ Small chunk sizes (10K rows) for processing")
    print("✅ Aggressive memory cleanup")
    print("✅ Timeout-resistant file reading")
    print("✅ Progress tracking every 100 chunks")
    
    print(f"\n🚀 SAFE EXECUTION:")
    print("# demo_caseid, demo_version, drug_caseid, drug_version = run_safe_deduplication()")
    
    # Uncomment to run safe analysis:
    demo_caseid, demo_version, drug_caseid, drug_version = run_safe_deduplication()

🛡️ TIMEOUT-SAFE FAERS DEDUPLICATION
Handles massive files without timeout errors!

🔧 SAFETY FEATURES:
✅ Tiny sample sizes (50-1000 rows) for analysis
✅ Small chunk sizes (10K rows) for processing
✅ Aggressive memory cleanup
✅ Timeout-resistant file reading
✅ Progress tracking every 100 chunks

🚀 SAFE EXECUTION:
# demo_caseid, demo_version, drug_caseid, drug_version = run_safe_deduplication()
🛡️ TIMEOUT-SAFE FAERS DEDUPLICATION

STEP 1: ANALYZING CLEANED DEMOGRAPHICS

🔍 SAFE ANALYSIS: CLEANED DEMOGRAPHICS
🔍 Quick check: faers_demographics_cleaned.csv
✅ File exists: 3.44 GB
📋 Reading header...
✅ Found 39 columns
📊 Reading tiny sample (50 rows)...
✅ Sample loaded: (50, 39)
🎯 Looking for deduplication columns...
✅ Found case ID: caseid
✅ Found version: caseversion

📋 Sample data:
     caseid  caseversion
0  10004694            2
1  10004820            6
2  10004862            2
3  10005135            2
4  10005138            2

📈 Versions found in sample: [2, 3, 4, 5, 6, 25, 29]

📊 SAFE DU

In [5]:
demo_result = deduplicate_small_file_fast(DEMO_CLEANED_PATH, DEMO_DEDUPE_PATH, 'caseid', 'caseversion', 'demographics')



🚀 FAST DEDUPLICATION: DEMOGRAPHICS
📊 Loading entire file...
✅ Loaded: 15,876,714 rows, 39 columns
🔄 Converting versions to numeric...
🎯 Keeping latest version per case...
💾 Saving deduplicated file...

✅ DEDUPLICATION COMPLETE!
📊 RESULTS:
  Original rows: 15,876,714
  Final rows: 6,775,056
  Removed: 9,101,658 (57.3%)
  Size: 3522.7 MB → 1508.3 MB (57.2% reduction)
  Output: /Users/deliciamagdaline/Desktop/Project/faers_menstrual_rag_project/data/processed_cleaned/faers_demographics_deduplicated.csv


Instead of keeping all the drugs a patient takes (like vitamins, blood pressure meds, etc.), this code only keeps the ONE drug that doctors think actually caused the bad reaction.

In [5]:
import pandas as pd
import numpy as np
import os
import gc
import warnings
warnings.filterwarnings('ignore')

# FILE PATHS
DRUG_INPUT_PATH = "/Users/deliciamagdaline/Desktop/Project/faers_menstrual_rag_project/data/processed/faers_drugs_combined_2020_2024.csv"
DRUG_PRIMARY_PATH = "/Users/deliciamagdaline/Desktop/Project/faers_menstrual_rag_project/data/processed_cleaned/faers_drugs_primary_suspects.csv"

def check_role_cod_column():
    """
    Check if ROLE_COD column exists and what values it contains
    """
    print("🔍 CHECKING ROLE_COD COLUMN")
    print("=" * 50)
    
    if not os.path.exists(DRUG_INPUT_PATH):
        print(f"❌ Drug file not found")
        return None
    
    try:
        # Check file size
        file_size_gb = os.path.getsize(DRUG_INPUT_PATH) / (1024**3)
        print(f"📁 Drug file size: {file_size_gb:.2f} GB")
        
        # Read small sample to check structure
        print(f"📊 Reading sample to check ROLE_COD...")
        sample_df = pd.read_csv(DRUG_INPUT_PATH, nrows=10000, low_memory=False)
        
        print(f"Sample: {sample_df.shape[0]:,} rows, {sample_df.shape[1]} columns")
        
        # Look for ROLE_COD column (or variations)
        role_col = None
        role_candidates = ['role_cod', 'rolecod', 'role_code', 'drug_role']
        
        for candidate in role_candidates:
            if candidate in sample_df.columns:
                role_col = candidate
                break
        
        if not role_col:
            print(f"❌ No ROLE_COD column found")
            print(f"Available columns: {list(sample_df.columns)}")
            return None
        
        print(f"✅ Found role column: {role_col}")
        
        # Show role distribution
        role_counts = sample_df[role_col].value_counts()
        print(f"\n📈 Role distribution in sample:")
        for role, count in role_counts.items():
            pct = (count / len(sample_df)) * 100
            description = {
                'PS': 'Primary Suspect',
                'SS': 'Secondary Suspect', 
                'C': 'Concomitant',
                'I': 'Interacting'
            }.get(role, 'Unknown')
            print(f"  {role}: {count:,} ({pct:.1f}%) - {description}")
        
        # Count primary suspects
        primary_count = (sample_df[role_col] == 'PS').sum()
        primary_pct = (primary_count / len(sample_df)) * 100
        
        print(f"\n🎯 PRIMARY SUSPECTS:")
        print(f"  Count in sample: {primary_count:,}")
        print(f"  Percentage: {primary_pct:.1f}%")
        print(f"  Estimated in full file: ~{(primary_pct/100) * file_size_gb:.1f} GB")
        
        return role_col
        
    except Exception as e:
        print(f"❌ Error checking ROLE_COD: {e}")
        return None

def filter_primary_suspects_chunked(role_col, chunk_size=50000):
    """
    Filter massive drug file to keep only Primary Suspect drugs
    Uses chunked processing for 8GB+ files
    """
    print(f"\n💊 FILTERING TO PRIMARY SUSPECTS ONLY")
    print("=" * 60)
    print(f"Strategy: Keep only ROLE_COD = 'PS' (Primary Suspect drugs)")
    print(f"Chunk size: {chunk_size:,} rows")
    
    try:
        # Create output directory
        os.makedirs(os.path.dirname(DRUG_PRIMARY_PATH), exist_ok=True)
        
        # Save header first
        print(f"📋 Setting up output file...")
        header_df = pd.read_csv(DRUG_INPUT_PATH, nrows=0)
        header_df.to_csv(DRUG_PRIMARY_PATH, index=False)
        
        # Process file in chunks
        print(f"🔄 Processing file in chunks...")
        
        total_processed = 0
        primary_kept = 0
        chunk_count = 0
        
        chunk_iterator = pd.read_csv(DRUG_INPUT_PATH, chunksize=chunk_size, low_memory=False)
        
        for chunk in chunk_iterator:
            chunk_count += 1
            
            # Filter to primary suspects only
            primary_chunk = chunk[chunk[role_col] == 'PS'].copy()
            
            # Save filtered chunk
            if not primary_chunk.empty:
                primary_chunk.to_csv(DRUG_PRIMARY_PATH, mode='a', header=False, index=False)
                primary_kept += len(primary_chunk)
            
            total_processed += len(chunk)
            
            # Progress update every 100 chunks
            if chunk_count % 100 == 0:
                gb_processed = (total_processed * 4) / (1024**3)
                print(f"    Processed {chunk_count} chunks, ~{gb_processed:.1f}GB")
                print(f"    Kept {primary_kept:,} primary suspects from {total_processed:,} total drugs...")
            
            # Memory cleanup
            del chunk, primary_chunk
            gc.collect()
        
        # Final statistics
        filtered_out = total_processed - primary_kept
        filter_rate = (filtered_out / total_processed) * 100
        
        # File size comparison
        input_size_gb = os.path.getsize(DRUG_INPUT_PATH) / (1024**3)
        output_size_gb = os.path.getsize(DRUG_PRIMARY_PATH) / (1024**3)
        size_reduction = ((input_size_gb - output_size_gb) / input_size_gb) * 100
        
        print(f"\n🎉 PRIMARY SUSPECT FILTERING COMPLETE!")
        print("=" * 60)
        print(f"📊 FILTERING RESULTS:")
        print(f"  Total input drugs: {total_processed:,}")
        print(f"  Primary suspects kept: {primary_kept:,}")
        print(f"  Secondary/other removed: {filtered_out:,}")
        print(f"  Filter rate: {filter_rate:.1f}%")
        print(f"  Input size: {input_size_gb:.2f} GB")
        print(f"  Output size: {output_size_gb:.2f} GB")
        print(f"  Size reduction: {size_reduction:.1f}%")
        print(f"  Output file: {DRUG_PRIMARY_PATH}")
        
        print(f"\n💡 RESULT:")
        print(f"Now you have ONE suspected culprit drug per case!")
        print(f"Perfect for analyzing drug → menstrual disturbance relationships!")
        
        return {
            'total_drugs': total_processed,
            'primary_suspects': primary_kept,
            'filter_rate': filter_rate,
            'size_reduction': size_reduction
        }
        
    except Exception as e:
        print(f"❌ Error during filtering: {e}")
        return None

def show_sample_primary_suspects():
    """
    Show sample of filtered primary suspect drugs
    """
    print(f"\n📋 SAMPLE OF PRIMARY SUSPECT DRUGS")
    print("=" * 50)
    
    if not os.path.exists(DRUG_PRIMARY_PATH):
        print(f"❌ Primary suspects file not found. Run filtering first.")
        return
    
    try:
        # Load sample
        sample_df = pd.read_csv(DRUG_PRIMARY_PATH, nrows=10)
        
        print(f"Sample from filtered file:")
        print(f"Rows: {sample_df.shape[0]}, Columns: {sample_df.shape[1]}")
        
        # Show key columns
        key_cols = ['caseid', 'drugname', 'role_cod']
        available_cols = [col for col in key_cols if col in sample_df.columns]
        
        if available_cols:
            print(f"\nSample data:")
            print(sample_df[available_cols].head())
        
        # Show unique drug names in sample
        if 'drugname' in sample_df.columns:
            unique_drugs = sample_df['drugname'].unique()
            print(f"\nSample primary suspect drugs:")
            for drug in unique_drugs[:10]:
                print(f"  - {drug}")
        
    except Exception as e:
        print(f"❌ Error showing sample: {e}")

def run_primary_suspect_filtering():
    """
    Complete workflow to filter drug file to primary suspects only
    """
    print("🎯 PRIMARY SUSPECT DRUG FILTERING")
    print("=" * 70)
    print("Keep only the ONE drug most likely to cause adverse events per case")
    
    # Step 1: Check if ROLE_COD exists
    role_col = check_role_cod_column()
    
    if not role_col:
        print("❌ Cannot proceed - no ROLE_COD column found")
        return None
    
    # Step 2: Ask for confirmation
    print(f"\n⚠️  READY TO FILTER 8.77 GB DRUG FILE")
    print(f"This will take 20-40 minutes but create much cleaner data")
    print(f"Ready to proceed? Uncomment line below:")
    print(f"# result = filter_primary_suspects_chunked('{role_col}')")
    
    return role_col

# 🚀 MAIN EXECUTION
if __name__ == "__main__":
    print("💊 PRIMARY SUSPECT DRUG FILTERING")
    print("=" * 70)
    print("Strategy: Keep only ROLE_COD = 'PS' drugs")
    print("Result: One suspected culprit drug per case")
    
    print(f"\n🎯 WHY THIS IS BETTER:")
    print("✅ Focuses on the actual problem drug")
    print("✅ Eliminates noise from unrelated medications")
    print("✅ Perfect for menstrual disturbance analysis")
    print("✅ Much smaller, cleaner dataset")
    
    print(f"\n🚀 TO START:")
    print("# role_col = run_primary_suspect_filtering()")
    
    # Uncomment to run:
    role_col = run_primary_suspect_filtering()
    result = filter_primary_suspects_chunked(role_col)
    show_sample_primary_suspects()

💊 PRIMARY SUSPECT DRUG FILTERING
Strategy: Keep only ROLE_COD = 'PS' drugs
Result: One suspected culprit drug per case

🎯 WHY THIS IS BETTER:
✅ Focuses on the actual problem drug
✅ Eliminates noise from unrelated medications
✅ Perfect for menstrual disturbance analysis
✅ Much smaller, cleaner dataset

🚀 TO START:
# role_col = run_primary_suspect_filtering()
🎯 PRIMARY SUSPECT DRUG FILTERING
Keep only the ONE drug most likely to cause adverse events per case
🔍 CHECKING ROLE_COD COLUMN
📁 Drug file size: 8.77 GB
📊 Reading sample to check ROLE_COD...
Sample: 10,000 rows, 23 columns
✅ Found role column: role_cod

📈 Role distribution in sample:
  SS: 4,431 (44.3%) - Secondary Suspect
  C: 4,353 (43.5%) - Concomitant
  PS: 1,152 (11.5%) - Primary Suspect
  I: 64 (0.6%) - Interacting

🎯 PRIMARY SUSPECTS:
  Count in sample: 1,152
  Percentage: 11.5%
  Estimated in full file: ~1.0 GB

⚠️  READY TO FILTER 8.77 GB DRUG FILE
This will take 20-40 minutes but create much cleaner data
Ready to proceed?

## Drug Names: This is a major task.

Normalize: Convert all drug names to lowercase.

Map to Generic Names: The same drug is reported under hundreds of brand names and misspellings. You need to map these to a standard generic name (the active ingredient). Use a drug dictionary like RxNorm for this. For example, "Advil," "Motrin," and "Ibuprofen" should all be mapped to "ibuprofen." for drug df 

In [7]:
!pip install pandas requests



python(4150) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


RXnorms

In [9]:
# 1. First, check what columns you have
import pandas as pd
df = pd.read_csv('/Users/deliciamagdaline/Desktop/Project/faers_menstrual_rag_project/data/processed_cleaned/faers_drugs_primary_suspects.csv', nrows=5)
print(df.columns)

# 2. Tell me what drug column name you have, and I'll help you run the optimized process

Index(['primaryid', 'caseid', 'drug_seq', 'role_cod', 'drugname', 'prod_ai',
       'val_vbm', 'route', 'dose_vbm', 'cum_dose_chr', 'cum_dose_unit',
       'dechal', 'rechal', 'lot_num', 'exp_dt', 'nda_num', 'dose_amt',
       'dose_unit', 'dose_form', 'dose_freq', 'year', 'quarter',
       'source_file'],
      dtype='object')


RXnorm


In [21]:
# ================================================================================
# COMPLETE FAERS DRUG STANDARDIZATION PIPELINE - FOR A SINGLE JUPYTER NOTEBOOK CELL
# ================================================================================
# This script contains all necessary classes and functions to perform the full
# FAERS drug name standardization workflow.
#
# Workflow:
# 1. Imports and Class Definition: All setup is at the top.
# 2. Helper Functions: The logic for each step of the pipeline.
# 3. Main Execution: The main block at the bottom runs the entire process.
#
# To Run: Simply execute this cell in your Jupyter Notebook.
# ================================================================================

# --- Imports ---
import pandas as pd
import numpy as np
import os
import time
import requests
import sqlite3
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

# ================================================================================
# CLASS DEFINITION: The tool for mapping drug names
# ================================================================================

class FAERSRxNormMapper:
    """
    Maps drug names to their standardized RxNorm counterparts using the NIH RxNav API.
    Caches results in a local SQLite database to improve performance and reduce API load.
    """
    
    def __init__(self, cache_db: str = "faers_rxnorm_cache.db"):
        """Initializes the mapper, sets up the API session, and connects to the cache."""
        self.base_url = "https://rxnav.nlm.nih.gov/REST"
        self.cache_db = cache_db
        self.cache_conn = None
        self._connect_to_cache()
        self.session = self._create_robust_session()
        print(f"FAERSRxNormMapper initialized. Cache located at: '{self.cache_db}'")

    def _connect_to_cache(self):
        """Establishes a connection to the SQLite database and creates the cache table."""
        try:
            self.cache_conn = sqlite3.connect(self.cache_db, check_same_thread=False)
            cursor = self.cache_conn.cursor()
            cursor.execute('''
                CREATE TABLE IF NOT EXISTS rxnorm_cache (
                    drug_name TEXT PRIMARY KEY,
                    standard_name TEXT,
                    rxcui TEXT,
                    score REAL,
                    mapping_method TEXT
                )
            ''')
            self.cache_conn.commit()
        except sqlite3.Error as e:
            print(f"Database error: {e}")
            raise

    def _create_robust_session(self):
        """Creates a requests.Session with automatic retries for resilience."""
        session = requests.Session()
        retries = Retry(total=5, backoff_factor=0.5, status_forcelist=[500, 502, 503, 504])
        adapter = HTTPAdapter(max_retries=retries)
        session.mount("https://", adapter)
        return session

    def _query_cache(self, drug_name: str):
        """Checks the local cache for a previously mapped drug name."""
        try:
            cursor = self.cache_conn.cursor()
            cursor.execute("SELECT standard_name, rxcui, score, mapping_method FROM rxnorm_cache WHERE drug_name = ?", (drug_name,))
            result = cursor.fetchone()
            if result:
                return {'standard_name': result[0], 'rxcui': result[1], 'score': result[2], 'mapping_method': result[3]}
        except sqlite3.Error as e:
            print(f"Cache query error for '{drug_name}': {e}")
        return None

    def _update_cache(self, drug_name: str, result: dict):
        """Stores a new mapping result in the local cache."""
        try:
            cursor = self.cache_conn.cursor()
            cursor.execute("INSERT OR REPLACE INTO rxnorm_cache (drug_name, standard_name, rxcui, score, mapping_method) VALUES (?, ?, ?, ?, ?)",
                           (drug_name, result.get('standard_name'), result.get('rxcui'), result.get('score'), result.get('mapping_method')))
            self.cache_conn.commit()
        except sqlite3.Error as e:
            print(f"Cache update error for '{drug_name}': {e}")

    def _query_rxnorm_api(self, drug_name: str):
        """Queries the RxNorm API to get the standardized name and RxCUI."""
        url = f"{self.base_url}/approximateTerm.json"
        params = {'term': drug_name, 'maxEntries': 1}
        try:
            response = self.session.get(url, params=params, timeout=10)
            response.raise_for_status()
            data = response.json()
            candidates = data.get('approximateGroup', {}).get('candidate', [])
            if candidates:
                best_match = candidates[0]
                return {'standard_name': best_match.get('rxcuiName'), 'rxcui': best_match.get('rxcui'), 'score': best_match.get('score'), 'mapping_method': 'approximateTerm'}
        except requests.exceptions.RequestException as e:
            print(f"API request failed for '{drug_name}': {e}")
        return None

    def get_standardized_drug_name(self, drug_name: str):
        """Public method to get a standardized drug name, using cache first."""
        if not drug_name or pd.isna(drug_name):
            return None
        
        cached_result = self._query_cache(drug_name)
        if cached_result:
            cached_result['mapping_method'] = 'cache'
            return cached_result
        
        api_result = self._query_rxnorm_api(drug_name)
        
        if api_result:
            self._update_cache(drug_name, api_result)
        else:
            self._update_cache(drug_name, {'mapping_method': 'api_fail'})
        
        time.sleep(0.03)  # Respect API rate limits
        return api_result

    def close_connection(self):
        """Closes the connection to the cache database."""
        if self.cache_conn:
            self.cache_conn.close()
            print("Cache connection closed.")

# ================================================================================
# PIPELINE HELPER FUNCTIONS
# ================================================================================

def create_sample_faers_file(file_path: str, num_rows: int = 50000):
    """Generates a sample FAERS drug file for demonstration."""
    if os.path.exists(file_path):
        print(f"Sample file '{file_path}' already exists. Skipping generation.")
        return
    print(f"Generating a sample FAERS file with {num_rows:,} rows at '{file_path}'...")
    drug_pool = ['ASPIRIN', 'LIPITOR', 'METFORMIN', 'LISINOPRIL', 'IBUPROFEN', 'ADVIL', 'TYLENOL', 'ATORVASTATIN', 'Glucophage', 'Prozac', 'Xanax', 'OxyContin', 'VITAMIN C', np.nan]
    p = np.linspace(1, 0.1, len(drug_pool)); p /= p.sum()
    data = {'primaryid': [f"CASE{100000 + i}" for i in range(num_rows)], 'drugname': np.random.choice(drug_pool, size=num_rows, p=p)}
    df = pd.DataFrame(data)
    df['caseid'] = df['primaryid']
    df.to_csv(file_path, index=False)
    print(f"Sample file '{file_path}' created successfully.")

def extract_unique_faers_drugs(file_path: str, chunksize: int = 50000):
    """Extracts unique drug names from the FAERS drug file."""
    print("\n" + "="*60 + "\nSTEP 1: EXTRACTING UNIQUE DRUGS\n" + "="*60)
    start_time = time.time()
    unique_drugs = set()
    for chunk in pd.read_csv(file_path, chunksize=chunksize, usecols=['drugname'], low_memory=False):
        unique_drugs.update(chunk['drugname'].dropna().str.strip().unique())
    print(f"Extraction Complete! Found {len(unique_drugs):,} unique drugs in {time.time() - start_time:.2f}s.")
    return pd.DataFrame({'drugname': sorted(list(unique_drugs))})

def calculate_drug_frequencies(file_path: str, unique_drugs_df: pd.DataFrame, chunksize: int = 50000):
    """Calculates the frequency of each unique drug."""
    print("\n" + "="*60 + "\nSTEP 2: CALCULATING DRUG FREQUENCIES\n" + "="*60)
    start_time = time.time()
    drug_counts = pd.Series(dtype=int)
    for chunk in pd.read_csv(file_path, chunksize=chunksize, usecols=['drugname'], low_memory=False):
        drug_counts = drug_counts.add(chunk['drugname'].dropna().str.strip().value_counts(), fill_value=0)
    unique_drugs_df['frequency'] = unique_drugs_df['drugname'].map(drug_counts).astype(int)
    unique_drugs_df = unique_drugs_df.sort_values('frequency', ascending=False).reset_index(drop=True)
    print(f"Frequency calculation complete in {time.time() - start_time:.2f}s.")
    print("Top 10 most frequent drugs:\n", unique_drugs_df.head(10).to_string())
    return unique_drugs_df

def map_unique_drugs_batch(unique_drugs_df: pd.DataFrame, start_from: int = 0, batch_size: int = 50):
    """Maps unique drugs to RxNorm with progress tracking and checkpointing."""
    print("\n" + "="*60 + "\nSTEP 3: MAPPING UNIQUE DRUGS TO RXNORM\n" + "="*60)
    mapper = FAERSRxNormMapper()
    for col in ['rxnorm_name', 'rxcui', 'mapping_score', 'mapping_method']:
        if col not in unique_drugs_df.columns:
            unique_drugs_df[col] = None
    total_drugs = len(unique_drugs_df)
    start_time = time.time()
    try:
        for idx, row in unique_drugs_df.iloc[start_from:].iterrows():
            if pd.notna(unique_drugs_df.loc[idx, 'rxcui']): continue
            result = mapper.get_standardized_drug_name(row['drugname'])
            if result:
                unique_drugs_df.loc[idx, ['rxnorm_name', 'rxcui', 'mapping_score', 'mapping_method']] = result.values()
            if (idx + 1) % batch_size == 0 or (idx + 1) == total_drugs:
                processed = idx + 1
                rate = processed / (time.time() - start_time) if (time.time() - start_time) > 0 else 0
                success_rate = unique_drugs_df['rxcui'].notna().sum() / processed * 100
                print(f"  Progress: {processed}/{total_drugs} ({processed/total_drugs:.1%}) | Success: {success_rate:.1f}% | Rate: {rate:.1f} drugs/sec")
    except KeyboardInterrupt:
        print("\nProcess interrupted. Saving progress...")
    finally:
        mapper.close_connection()
    final_file = 'faers_unique_drugs_mapped_final.csv'
    unique_drugs_df.to_csv(final_file, index=False)
    print(f"\nMapping complete! Final results saved to: {final_file}")
    return unique_drugs_df

def apply_mappings_to_faers(original_file: str, mapping_df: pd.DataFrame, output_file: str, chunksize: int = 50000):
    """Applies the RxNorm mappings back to the full FAERS file."""
    print("\n" + "="*60 + "\nSTEP 4: APPLYING MAPPINGS TO FULL FILE\n" + "="*60)
    mapping_df.set_index('drugname', inplace=True)
    is_first_chunk = True
    for chunk in pd.read_csv(original_file, chunksize=chunksize, low_memory=False):
        # Clean drugname before merging
        chunk['drugname_clean'] = chunk['drugname'].str.strip()
        merged_chunk = chunk.merge(mapping_df, left_on='drugname_clean', right_index=True, how='left').drop(columns=['drugname_clean'])
        
        mode = 'w' if is_first_chunk else 'a'
        header = is_first_chunk
        merged_chunk.to_csv(output_file, index=False, mode=mode, header=header)
        is_first_chunk = False
    print(f"Processing Complete! Full file with mappings saved to: {output_file}")

# ================================================================================
# MAIN EXECUTION BLOCK
# ================================================================================

# --- Configuration ---
# Updated to use the specific file path provided by the user.
FAERS_FILE = "/Users/deliciamagdaline/Desktop/Project/faers_menstrual_rag_project/data/processed_cleaned/faers_drugs_primary_suspects.csv"
FINAL_OUTPUT_FILE = "/Users/deliciamagdaline/Desktop/Project/faers_menstrual_rag_project/data/processed_cleaned/faers_drug_rxnorm_standardized.csv"

# --- Run Pipeline ---
print("="*80)
print("      STARTING FAERS DRUG STANDARDIZATION PIPELINE      ")
print(f"      Input File: {FAERS_FILE}      ")
print("="*80)

# The sample file creation is commented out as we are now using a real file path.
# create_sample_faers_file(FAERS_FILE) 

# Step 1: Extract unique drugs
unique_drugs_df = extract_unique_faers_drugs(FAERS_FILE)

# Step 2: Calculate frequencies (sorts by most common for efficient mapping)
unique_drugs_df = calculate_drug_frequencies(FAERS_FILE, unique_drugs_df)

# Step 3: Map to RxNorm (this is the most time-consuming step)
mapped_drugs_df = map_unique_drugs_batch(unique_drugs_df)

# Step 4: Apply mappings back to the original full file
apply_mappings_to_faers(
    original_file=FAERS_FILE,
    mapping_df=mapped_drugs_df,
    output_file=FINAL_OUTPUT_FILE
)

print("\n\n✅ PIPELINE COMPLETE!")
print(f"The final standardized file is available at: '{FINAL_OUTPUT_FILE}'")
print("You can now proceed with analysis on this file.")
print("="*80)



      STARTING FAERS DRUG STANDARDIZATION PIPELINE      
      Input File: /Users/deliciamagdaline/Desktop/Project/faers_menstrual_rag_project/data/processed_cleaned/faers_drugs_primary_suspects.csv      

STEP 1: EXTRACTING UNIQUE DRUGS
Extraction Complete! Found 41,649 unique drugs in 27.20s.

STEP 2: CALCULATING DRUG FREQUENCIES
Frequency calculation complete in 38.87s.
Top 10 most frequent drugs:
                                drugname  frequency
0                              DUPIXENT     625122
1                                ZANTAC     594274
2                                HUMIRA     394674
3                              REVLIMID     319190
4                              COSENTYX     219804
5                             OXYCONTIN     199872
6  PROACTIV MD ADAPALENE ACNE TREATMENT     190464
7                               REPATHA     173372
8                              ENTRESTO     151206
9                            XELJANZ XR     145826

STEP 3: MAPPING UNIQUE DRUGS TO R

2025-08-03 05:31:43,815 - WARNING - Retrying (Retry(total=4, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ReadTimeoutError("HTTPSConnectionPool(host='rxnav.nlm.nih.gov', port=443): Read timed out. (read timeout=10)")': /REST/approximateTerm.json?term=BREVIBLOC&maxEntries=1


  Progress: 9550/41649 (22.9%) | Success: 96.4% | Rate: 2.0 drugs/sec
  Progress: 9600/41649 (23.0%) | Success: 96.4% | Rate: 2.0 drugs/sec


2025-08-03 05:48:27,865 - WARNING - Retrying (Retry(total=4, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ReadTimeoutError("HTTPSConnectionPool(host='rxnav.nlm.nih.gov', port=443): Read timed out. (read timeout=10)")': /REST/approximateTerm.json?term=Dutasteride+Soft+Capsules&maxEntries=1


  Progress: 9650/41649 (23.2%) | Success: 96.4% | Rate: 1.7 drugs/sec
  Progress: 9700/41649 (23.3%) | Success: 96.3% | Rate: 1.7 drugs/sec


...

  Progress: 24200/41649 (58.1%) | Success: 95.8% | Rate: 0.9 drugs/sec
  Progress: 24250/41649 (58.2%) | Success: 95.8% | Rate: 0.9 drugs/sec


2025-08-03 11:58:22,167 - WARNING - Retrying (Retry(total=4, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ReadTimeoutError("HTTPSConnectionPool(host='rxnav.nlm.nih.gov', port=443): Read timed out. (read timeout=10)")': /REST/approximateTerm.json?term=SODIUM+PYROPHOSPHATE&maxEntries=1


  Progress: 24300/41649 (58.3%) | Success: 95.8% | Rate: 0.9 drugs/sec
  Progress: 24350/41649 (58.5%) | Success: 95.8% | Rate: 0.9 drugs/sec
  Progress: 24400/41649 (58.6%) | Success: 95.8% | Rate: 0.9 drugs/sec
  Progress: 24450/41649 (58.7%) | Success: 95.8% | Rate: 0.9 drugs/sec
  Progress: 24500/41649 (58.8%) | Success: 95.8% | Rate: 0.9 drugs/sec
  Progress: 24550/41649 (58.9%) | Success: 95.8% | Rate: 0.9 drugs/sec
  Progress: 24600/41649 (59.1%) | Success: 95.8% | Rate: 0.9 drugs/sec
  Progress: 24650/41649 (59.2%) | Success: 95.8% | Rate: 0.9 drugs/sec
  Progress: 24700/41649 (59.3%) | Success: 95.8% | Rate: 0.9 drugs/sec
  Progress: 24750/41649 (59.4%) | Success: 95.8% | Rate: 0.9 drugs/sec
  Progress: 24800/41649 (59.5%) | Success: 95.8% | Rate: 0.9 drugs/sec
  Progress: 24850/41649 (59.7%) | Success: 95.8% | Rate: 0.9 drugs/sec
  Progress: 24900/41649 (59.8%) | Success: 95.8% | Rate: 0.9 drugs/sec
  Progress: 24950/41649 (59.9%) | Success: 95.8% | Rate: 0.9 drugs/sec
  Prog

2025-08-03 12:08:58,913 - WARNING - Retrying (Retry(total=4, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ReadTimeoutError("HTTPSConnectionPool(host='rxnav.nlm.nih.gov', port=443): Read timed out. (read timeout=10)")': /REST/approximateTerm.json?term=Pregabalin+beta+75+mg+Hartkapseln&maxEntries=1


  Progress: 25200/41649 (60.5%) | Success: 95.8% | Rate: 0.9 drugs/sec
  Progress: 25250/41649 (60.6%) | Success: 95.8% | Rate: 0.9 drugs/sec
  Progress: 25300/41649 (60.7%) | Success: 95.8% | Rate: 0.9 drugs/sec
  Progress: 25350/41649 (60.9%) | Success: 95.8% | Rate: 0.9 drugs/sec
  Progress: 25400/41649 (61.0%) | Success: 95.8% | Rate: 0.9 drugs/sec
  Progress: 25450/41649 (61.1%) | Success: 95.8% | Rate: 0.9 drugs/sec
  Progress: 25500/41649 (61.2%) | Success: 95.9% | Rate: 0.9 drugs/sec
  Progress: 25550/41649 (61.3%) | Success: 95.9% | Rate: 0.9 drugs/sec
  Progress: 25600/41649 (61.5%) | Success: 95.9% | Rate: 0.9 drugs/sec
  Progress: 25650/41649 (61.6%) | Success: 95.9% | Rate: 0.9 drugs/sec
  Progress: 25700/41649 (61.7%) | Success: 95.9% | Rate: 0.9 drugs/sec
  Progress: 25750/41649 (61.8%) | Success: 95.9% | Rate: 0.9 drugs/sec
  Progress: 25800/41649 (61.9%) | Success: 95.9% | Rate: 0.9 drugs/sec
  Progress: 25850/41649 (62.1%) | Success: 95.9% | Rate: 0.9 drugs/sec
  Prog